In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import time
import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [53]:
class GRU_RNN_TWO(nn.Module):
        
    def __init__(self,input_size, hidden_size, num_of_layers, num_alpha_embedding, num_beta_embedding, output_size, dropout):
        
        super(GRU_RNN_TWO, self).__init__()
        
        self.input_size = input_size
        self.num_alpha_embedding = num_alpha_embedding
        self.num_beta_embedding = num_beta_embedding        
        
        self.hidden_0 = nn.GRU(  
                    input_size=input_size,
                    hidden_size=hidden_size,
                    num_layers=num_of_layers,
                    batch_first=True,
                    dropout=dropout
        )
        
        self.hidden_1 = nn.GRU(  
                    input_size=input_size + num_alpha_embedding + num_beta_embedding,
                    hidden_size=hidden_size,
                    num_layers=num_of_layers,
                    batch_first=True,
                    dropout=dropout 
        )
    
        
        self.out_alpha = nn.Linear(hidden_size, num_alpha_embedding)
        self.out_beta = nn.Linear(hidden_size, num_beta_embedding)
        
        self.relu_alpha = nn.ReLU()
        self.relu_beta = nn.ReLU()

        self.reg_alpha = nn.Linear(num_alpha_embedding, 1)
        self.reg_beta = nn.Linear(num_beta_embedding, 1)

        
        self.out_action = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):

        # parameters estimation
        output_0, hn_0 = self.hidden_0(x)
        
        output_alpha = self.out_alpha(output_0)
        output_alpha = self.relu_alpha(output_alpha)
        
        output_beta = self.out_beta(output_0)
        output_beta = self.relu_beta(output_beta)
        
        cont_output_alpha = self.reg_alpha(output_alpha)
        cont_output_beta = self.reg_beta(output_beta)
    
        # concat input 
        input_1 = torch.concat([x[0],output_alpha[0],output_beta[0]],dim=1) # cat
        input_1 = input_1.reshape(1,x.shape[1],self.input_size + self.num_alpha_embedding
                                  + self.num_beta_embedding)
        
        # action predication
        output_action, hn_1 = self.hidden_1(input_1)
        output_action = self.out_action(output_action)
        output_action = F.softmax(output_action,dim=-1)
        
        output_dis = [output_alpha, output_beta]
        output_cont = [cont_output_alpha, cont_output_beta]

        return output_dis, output_cont, output_action, hn_0, hn_1
    

In [43]:
torch.arange(0, 5)%3
F.one_hot(torch.arange(0, 1) , num_classes=2)

tensor([[1, 0]])

In [54]:
class behavior_dataset(Dataset):
    """         
    """
    def __init__(self,dataframe):
        
        # action one hot transformation 
        action = np.array(dataframe['action'])
        if np.all(action == action[0]):
            action = np.append(action,(1-action[0]))
            action = torch.tensor((action).reshape(len(dataframe) + 1),dtype=int)
            action_onehot = nn.functional.one_hot(action, len(action.unique()))
            # delete last one
            action_onehot = action_onehot[:-1]
        else:
            action = torch.tensor((action).reshape(len(dataframe)),dtype=int)
            action_onehot = nn.functional.one_hot(action, len(action.unique()))
        
        # reward
        reward = torch.tensor((np.array(dataframe['reward'])).reshape(len(dataframe)),dtype=int)
        
        # state one hot transformation
        state = np.array(dataframe['state'])
        state_shift = np.insert(state,-1,0)[1:]
        state_shift = torch.tensor((state_shift).reshape(len(dataframe)),dtype=int)
        state_onehot = nn.functional.one_hot(state_shift, 12)


        # concatinating reward and action
        reward_action_state = torch.cat([reward[ :, np.newaxis], action_onehot, state_onehot],1)
        
        # adding dummy zeros to the beginning and ignoring the last one
        reward_action_state_shift = nn.functional.pad(reward_action_state,[0,0,1,0])[:-1]
        
        # make the first trial in every block to zeros
        n_blocks = int(len(dataframe)/25) 
        reward_action_state_shift.reshape(n_blocks,25,INPUT_SIZE)[:,0,:] = torch.zeros(size=(n_blocks,INPUT_SIZE))

        # parameters one hot transformation 
        dis_alpha = torch.tensor((np.array(dataframe['alpha_categorical'])).reshape(len(dataframe)),dtype=int)
        dis_alpha = dis_alpha.type(dtype=torch.float32)
        
        # parameters one hot transformation 
        dis_beta = torch.tensor((np.array(dataframe['beta_categorical'])).reshape(len(dataframe)),dtype=int)
        dis_beta = dis_beta.type(dtype=torch.float32)

        cont_alpha = torch.tensor((np.array(dataframe['alpha'])).reshape(len(dataframe)),dtype=torch.float32)
        cont_beta = torch.tensor((np.array(dataframe['beta'])).reshape(len(dataframe)),dtype=torch.float32)
        
        # network input 
        x = reward_action_state_shift

        # network output 
        y = torch.cat([action_onehot, dis_alpha[ :, np.newaxis],  dis_beta[ :, np.newaxis], 
                                      cont_alpha[ :, np.newaxis], cont_beta[ :, np.newaxis]
                      ],1)
  
        self.x = x.type(dtype=torch.float32)
        self.y = y.type(dtype=torch.float32)
        self.len = len(dataframe)

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
  
    def __len__(self):
        return self.len    
    
class merge_behavior_dataset(Dataset):
    """ 
    Merge Dataset of each agent to one dataset
    
    Args:
        dataset_list: list of Dataset of all agent 
        n_trials: num_of_trials each agent was simulated
        
    Returns: 
        torch Dataset:
        x: [reward_(t-1) , action_(t-1), state_(t-1)] all agents
        y: [action_t, parameter embedding] all agents
        
    """
    def __init__(self, dataset_list, n_trials):
        X = []
        Y = []
        for agent in dataset_list:
            for i in range(n_trials):
                X.append(agent[i][0])
                Y.append(agent[i][1])
                
        self.x = torch.stack(X).type(dtype=torch.float32)
        self.y = torch.stack(Y).type(dtype=torch.float32)
        self.len = len(X)   
    
    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
  
    def __len__(self):
        return self.len    

In [55]:
def train_model_two(net, train_loader, val_loader ,epochs, name):
        
    min_loss_t = 100
    min_loss_v = 100
        
    # array to track loss 
    train_loss_array , val_loss_array = [], []
    
    # move net to GPU
    net.to(device)

    # Use Adam optimizer
    optimizer = optim.Adam(net.parameters(), lr=0.001) 
    
    # start timer
    start_time = time.time()   
    
    # Loop over epochs 
    for i in range(epochs):        
        # Randomize train batch example 
        train_loader = random.sample(list(train_loader), len(train_loader))
      
        running_loss = 0
        
        run_action, run_d_alpha ,run_d_beta = 0,0,0
        run_c_alpha, run_c_beta  = 0,0 
        # Loop over training batches
        for j, (X, y_true) in enumerate(train_loader):
    
            X, y_true = X.to(device), y_true.to(device) # move to GPU
            X = X.reshape(1,X.shape[0],INPUT_SIZE) # reshape to  1 x trials x input_size
            
            optimizer.zero_grad()  # zero the gradient buffers
            
            y_hat_dis, y_hat_cont, y_hat_action, _, _  = net(X) # forward pass
            
            y_hat_action = (y_hat_action.view(-1, num_of_action)) # Reshape to (SeqLen x Batch, OutputSize)
            y_hat_dis_alpha = (y_hat_dis[0].view(-1, num_alpha_embedding)) # Reshape to (SeqLen x Batch, OutputSize)
            y_hat_dis_beta = (y_hat_dis[1].view(-1, num_beta_embedding)) # Reshape to (SeqLen x Batch, OutputSize)

            
            y_hat_dis = [y_hat_dis_alpha,y_hat_dis_beta]
 
            y_hat_cont_alpha = (y_hat_cont[0].view(-1)) # Reshape to (SeqLen x Batch, OutputSize)
            y_hat_cont_beta = (y_hat_cont[1].view(-1)) # Reshape to (SeqLen x Batch, OutputSize)

            
            y_hat_cont = [y_hat_cont_alpha,y_hat_cont_beta]

            loss, l_action, l_dis, l_cont = multi_loss(y_hat_action, y_hat_dis, y_hat_cont, y_true) # compute loss
                                                                                           

            loss.backward() # backprop the loss
            optimizer.step() # update the weights 
            running_loss += loss.item()
            
            run_action += l_action.item()
        
            run_d_alpha += l_dis[0].item()
            run_d_beta += l_dis[1].item()

            
            run_c_alpha += l_cont[0].item()
            run_c_beta += l_cont[1].item()

            
        train_loss_array.append(running_loss/len(train_loader))
        loss = eval_net_two(net,val_loader)
        val_loss_array.append(loss)
        
        print('================')
        print('loss BCE action',(run_action/len(train_loader)))
        
        print('================')
        print('loss CE alpha',(run_d_alpha/len(train_loader)))
        print('loss CE beta',(run_d_beta/len(train_loader)))

        
        print('================')
        print('loss mse alpha',(run_c_alpha/len(train_loader)))
        print('loss mse beta',(run_c_beta/len(train_loader)))
    
        
        if train_loss_array[i] <= min_loss_t:
            checkpoint = {'epoch':i+1,'model_state':net.state_dict(),
                          'optim_state':optimizer.state_dict(),'loss':train_loss_array[i]}
            torch.save(checkpoint,f'checkpoint_best_train_{name}.pth')

            min_loss_t = train_loss_array[i]

        if val_loss_array[i] <= min_loss_v:
            checkpoint = {'epoch':i+1,'model_state':net.state_dict(),
                          'optim_state':optimizer.state_dict(),'loss':val_loss_array[i]}
            torch.save(checkpoint,f'checkpoint_best_val_{name}.pth')

            min_loss_v = val_loss_array[i]

        print('Step {}, Train Loss {:0.4f}, Val Loss {:0.4f}, Time {:0.1f}s'.format(i+1,
                                                                                    train_loss_array[i],
                                                                                    val_loss_array[i],
                                                                                    time.time() - start_time))

        net.train()

            
    return net, train_loss_array , val_loss_array 

In [56]:
path = f'../data/artificial_trainset_2000_non_stat.csv'
df = pd.read_csv(path)
s = 0
e = 200
cur_df = df.iloc[s:e]
cur_df = cur_df.reset_index()
state = np.array(cur_df['state'])
print(state)
print(len(state))
new_state = np.insert(state,-1,0)[1:]
print(new_state)
print(len(new_state))
#state = torch.tensor((state).reshape(len(dataframe)),dtype=int)
#state_onehot = nn.functional.one_hot(state, 12)


[ 0  1  1  2  8  3 10  2  8  3  7  9  7  5 10  2  7 11  2  8  9  3  9 11
  5 11  9  3  1  5  9  8  7  7  1  3 10  6 10 11  6 10  3  4  8 10  4  8
  8  9  1  1  5  3  8  1  8  8  9  8  4  5 11  8  7  2  7  2  0  9  9  7
 11  8  1  9  4  0  9  4  8 10 10  7  0  0  2  4  9  7  8  2  4 11 11  5
  4  0  5  3  6  8  1 10  5 10  4  7  3  3 11 11  1  5 10 10  4  2  7  2
  6  3 11  5  9  3  5  0 11  2  0  9  9  4  1  9 11  2 10 11 11  3  7  9
  6  7  7  8  8  2  8  7  3 11  1  0  4 11  8  6  8  9  3  4  8 10  3 11
  4  0  2 10  4  3  6  8  5 11  3  5  3  1  7  3 10  4  4  7  7  0 11  1
  7 10  2  6  9  1 10  7]
200
[ 1  1  2  8  3 10  2  8  3  7  9  7  5 10  2  7 11  2  8  9  3  9 11  5
 11  9  3  1  5  9  8  7  7  1  3 10  6 10 11  6 10  3  4  8 10  4  8  8
  9  1  1  5  3  8  1  8  8  9  8  4  5 11  8  7  2  7  2  0  9  9  7 11
  8  1  9  4  0  9  4  8 10 10  7  0  0  2  4  9  7  8  2  4 11 11  5  4
  0  5  3  6  8  1 10  5 10  4  7  3  3 11 11  1  5 10 10  4  2  7  2  6
  3 11  5  9  3  5  0

In [57]:
def eval_net_two(net, val_loader):
    
    with torch.no_grad():
        net.eval()
        running_loss = 0
        for j, (X, y_true) in enumerate(val_loader):
            
            X, y_true = X.to(device), y_true.to(device) # move to GPU
            X = X.reshape(1,X.shape[0],INPUT_SIZE) # reshape to  1 x trials x input_size
            
            y_hat_dis, y_hat_cont, y_hat_action, _, _  = net(X) # forward pass
            
            y_hat_action = (y_hat_action.view(-1, num_of_action)) # Reshape to (SeqLen x Batch, OutputSize)
            y_hat_dis_alpha = (y_hat_dis[0].view(-1, num_alpha_embedding)) # Reshape to (SeqLen x Batch, OutputSize)
            y_hat_dis_beta = (y_hat_dis[1].view(-1, num_beta_embedding)) # Reshape to (SeqLen x Batch, OutputSize)

            
            y_hat_dis = [y_hat_dis_alpha,y_hat_dis_beta]
 
            y_hat_cont_alpha = (y_hat_cont[0].view(-1)) # Reshape to (SeqLen x Batch, OutputSize)
            y_hat_cont_beta = (y_hat_cont[1].view(-1)) # Reshape to (SeqLen x Batch, OutputSize)

            
            y_hat_cont = [y_hat_cont_alpha,y_hat_cont_beta]

            loss, l_action, l_dis, l_cont = multi_loss(y_hat_action, y_hat_dis, y_hat_cont, y_true) # compute loss
            
            running_loss += loss.item()
    
    return (running_loss/len(val_loader))
        

In [58]:
path = f'../data/artificial_trainset_2000_non_stat.csv'
df = pd.read_csv(path)

# define constant 
num_of_action = df['action'].nunique()
num_of_trials = df['trial'].nunique()
num_of_agents = df['agent'].nunique()
num_alpha_embedding = df['alpha_categorical'].nunique()
num_beta_embedding = df['beta_categorical'].nunique()

# netowrk input and output dimension  
INPUT_SIZE = 1 + num_of_action + 12
OUTPUT_SIZE = num_of_action

# train val test split 
n_agent_train = int(0.8*num_of_agents)
n_agent_val = int(0.2*num_of_agents)

all_data = []
for i in range(num_of_agents):
    s = i*num_of_trials
    e = (i+1)*num_of_trials
    cur_df = df.iloc[s:e]
    cur_df = cur_df.reset_index()
    all_data.append([i,behavior_dataset(cur_df)])
    
random.shuffle(all_data)
all_data = np.array(all_data)
train_dataset = all_data[:n_agent_train,1]
train_dataset = merge_behavior_dataset(train_dataset,num_of_trials)

val_dataset = all_data[n_agent_train:,1]
val_dataset = merge_behavior_dataset(val_dataset,num_of_trials)
val_agents = np.array([all_data[i,0] for i in range(n_agent_train,num_of_agents)])
    
print('num_of_trials:',num_of_trials)
print('num_of_agents:',num_of_agents)
print('num_alpha_embedding:',num_alpha_embedding)
print('num_beta_embedding:',num_beta_embedding)
print('train_size:', n_agent_train*num_of_trials)
print('train_size:', train_dataset[:][0].shape)
print('val_size:', n_agent_val*num_of_trials)
print('val_size:', val_dataset[:][0].shape)
# print('val_agents',val_agents)

num_of_trials: 200
num_of_agents: 2000
num_alpha_embedding: 5
num_beta_embedding: 5
train_size: 320000
train_size: torch.Size([320000, 15])
val_size: 80000
val_size: torch.Size([80000, 15])


In [31]:
df

,agent,block,trial,card_0,card_1,action,reward,state,state_onehot,alpha,beta,alpha_categorical,beta_categorical
0,0,0,0,1,0,1,0,3,[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.],0.569060,3.711152,2,1
1,0,0,1,2,1,1,1,7,[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.],0.569060,3.711152,2,1
2,0,0,2,1,0,0,1,3,[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.],0.569060,3.711152,2,1
3,0,0,3,3,2,0,1,11,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.],0.569060,3.711152,2,1
4,0,0,4,2,1,1,0,7,[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.],0.569060,3.711152,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,7,195,0,3,1,0,2,[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.],0.921076,2.638504,4,1
399996,1999,7,196,2,1,0,1,7,[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.],0.921076,2.638504,4,1
399997,1999,7,197,3,1,0,1,10,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.],0.921076,2.638504,4,1
399998,1999,7,198,2,0,1,1,6,[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.],0.921076,2.638504,4,1


In [34]:
#train_dataset[:][0][1]
train_dataset[:][0][2]

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [59]:
def multi_loss(y_hat_action, y_hat_dis, y_hat_cont, y_true):
    
    # slice true action and true parameters embedding 
    y_true_action = y_true[:,:2]
    
    y_true_dis_alpha = torch.flatten(y_true[:,2])
    y_true_dis_alpha = y_true_dis_alpha.type(dtype=torch.LongTensor).to(device)
    
    y_true_dis_beta = torch.flatten(y_true[:,3])
    y_true_dis_beta = y_true_dis_beta.type(dtype=torch.LongTensor).to(device)
    
    y_true_cont_alpha =  y_true[:,4]
    y_true_cont_beta =  y_true[:,5]
    
    # define losses
    criterion0 = nn.BCELoss()
    criterion1 = nn.CrossEntropyLoss()
    criterion2 = nn.MSELoss()
    
    l_action = criterion0(y_hat_action, y_true_action)
    
    l_dis_alpha = criterion1(y_hat_dis[0], y_true_dis_alpha)
    l_dis_beta = criterion1(y_hat_dis[1], y_true_dis_beta)
    
    l_cont_alpha = criterion2(y_hat_cont[0],y_true_cont_alpha)
    l_cont_beta = criterion2(y_hat_cont[1],y_true_cont_beta)
    
    # combine losses
    #total_loss = 1*l_action + 0.2*l_dis_alpha  +  0.2*l_dis_beta  + 6*l_cont_alpha  + 0.1*l_cont_beta 
    #total_loss = 1*l_action + 0.4*l_dis_alpha  +  0.5*l_dis_beta  + 6*l_cont_alpha  + 0.1*l_cont_beta  #7
    #total_loss = 1*l_action + 0.4*l_dis_alpha  +  0.45*l_dis_beta  + 6.5*l_cont_alpha  + 0.1*l_cont_beta  #8
    total_loss = 1*l_action + 0.35*l_dis_alpha  +  0.45*l_dis_beta  + 7*l_cont_alpha  + 0.1*l_cont_beta  #9
                
    return total_loss, 1*l_action, [0.35*l_dis_alpha,  0.45*l_dis_beta ], [7*l_cont_alpha, 0.1*l_cont_beta] 
                                   

In [9]:
train_dataset[0]

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 tensor([1.0000, 0.0000, 1.0000, 0.0000, 0.3476, 1.4063]))

In [60]:
aBatch = 1000
aHidden = 32

train_loader = DataLoader(train_dataset,shuffle=False,batch_size=aBatch)
val_loader = DataLoader(val_dataset,shuffle=False,batch_size=aBatch)
rnn = GRU_RNN_TWO(
              input_size=INPUT_SIZE,
              hidden_size=aHidden,
              num_of_layers=1,
              num_alpha_embedding=num_alpha_embedding,
              num_beta_embedding=num_beta_embedding,
              output_size=OUTPUT_SIZE,
              dropout=0.2
             ) 

rnn, loss_train, loss_val = train_model_two(rnn, train_loader, val_loader, 200, 9) 

loss BCE action 0.6780169371515512
loss CE alpha 0.5807486806996167
loss CE beta 0.9794085747562349
loss mse alpha 0.7428621260449291
loss mse beta 1.5723085119388998
Step 1, Train Loss 4.5533, Val Loss 3.7993, Time 18.7s
loss BCE action 0.6739369327202439
loss CE alpha 0.57486053686589
loss CE beta 1.0207731917500495
loss mse alpha 0.5867173779988661
loss mse beta 0.847118693497032
Step 2, Train Loss 3.7034, Val Loss 3.7078, Time 35.7s
loss BCE action 0.6732162581756711
loss CE alpha 0.5746967158280313
loss CE beta 0.9650489155203104
loss mse alpha 0.586731931567192
loss mse beta 0.8416957295034081
Step 3, Train Loss 3.6414, Val Loss 3.6637, Time 54.4s
loss BCE action 0.669904931075871
loss CE alpha 0.5739821156486868
loss CE beta 0.9309854963794351
loss mse alpha 0.5810561483260244
loss mse beta 0.8371499628759921
Step 4, Train Loss 3.5931, Val Loss 3.6192, Time 75.0s
loss BCE action 0.6635437130928039
loss CE alpha 0.5738034818321467
loss CE beta 0.9019699323922395
loss mse alpha 0.